In [13]:
#Given a length of an RNA sequence of which each character can be A,C,G,U, for each possible sequence of that length,
#find the total number of possible structures (excluding steric constraints due to hairpins etc.)
#Only constraint is that in each structure, each base pair can only bind to one other base pair
#(and base pairs can only bind if they're in the set {AU, CG, GU})

#The functions numPossStructs and numPossStructsFaster take as an argument the length of the sequence, 
#and return the average number of structures for any sequence of that length (by directly enumerating all possible
#sequences of a given length)

import numpy as np
import math

def getSeqNum(sequence): #Define the identity of a sequence by the number of As, Cs, Gs, Us it has
    nAs = sum([1 for i in sequence if i==0])
    nCs = sum([1 for i in sequence if i==1])
    nGs = sum([1 for i in sequence if i==2])
    nUs = sum([1 for i in sequence if i==3])
    return((nAs,nCs,nGs,nUs))
    
def countStructures(sequence,nStructuresVec,lenSeqStart):
    #For a given sequence, use the tabulated results in nStructuresVec to count the number of possible structures it can form
    #If the sequence (of length lenSeq) is actually a subsequence of a longer sequence (of length lenSeqStart),
    #there are factorial((lenSeqStart - lenSeq)/2 + 1) different ways of choosing the previous base pairs as well 
    #as the next base pair, so we discount such sequences by that factor.
    
    lenSeq = len(sequence)
    if lenSeq <= 1:
        return(0,nStructuresVec)

    seqNum = getSeqNum(sequence)

    if nStructuresVec[seqNum] != -1.: #-1 means we haven't yet examined this sequence
        return(nStructuresVec[seqNum],nStructuresVec)
    
    allowedBPs = [[0,3],[1,2],[2,1],[2,3],[3,0],[3,2]]
    #A can bind to U, C to G, G to C or U, U to A or G.
    
    #if we haven't yet done this sequence, go through the sequence, and once you find a base pair, 
    #add one and the number of structures for the resulting sequence not including the nts that comprised that base pair
    nStructuresVec[seqNum] = 0 #initialize to zero (rather than -1 which says we haven't done it yet)
    for j in range(lenSeq-1): #for each possible base pair j,k
        for k in range(j+1,lenSeq): 
            if [sequence[j],sequence[k]] in allowedBPs:
                #Add one (downweighted by the number of different ways of getting to this base pair given the 
                #number of previous base pairs -- so e.g. if I previously considered a base pair CG and now am considering
                #a base pair AU, I would be double counting since I could have also previously considered AU and now 
                #be considering CG)
                nStructuresVec[seqNum] += 1./(math.factorial((lenSeqStart-lenSeq)/2+1))
                
                #consider the sequence not including the nts that comprised that base pair
                noBPSeq = [sequence[i] for i in range(lenSeq) if (i != j and i != k)] 
                numOtherStructures,nStructuresVec = countStructures(noBPSeq,nStructuresVec,lenSeqStart)
                nStructuresVec[seqNum] += numOtherStructures
                
    return(nStructuresVec[seqNum],nStructuresVec)


def numPossStructs(lenSeq): #simplest way of doing this. Actually enumerate through each possible structure             
    
    def makeNextSeq(currSeq): #enumerate through the possible sequence space.
        #0 = A, 1 = C, 2 = G, 3 = U.
        if currSeq == [3]*len(currSeq): #if we've reached the last sequence in this enumeration scheme
            return([0]*len(currSeq)) #make the enumeration cyclical

        ind = len(currSeq) - 1 #index of ntd to change
        while currSeq[ind] == 3 and ind >= 0:
            ind -= 1
        currSeq[ind] += 1
        currSeq[ind+1:] = [0 for i in currSeq[ind+1:]]
        return(currSeq)
    
    chars = 'ACGU'
    #for each possible sequence of length lenSeq count the total number of possible structures 
    #(i.e. the number of non-overlapping pairs)
    
    nStructuresVec = np.zeros([lenSeq+1]*4) - 1 #index 0 is number of As, ... index 3 is number of Us
    #keep track for each possible sequence the number of possible structures it can form
    
    nStructures = 0
    currSeq = [0]*lenSeq
    ind = lenSeq - 1
    for i in range(4**lenSeq): #for each sequence i
        nStructuresForSeq,nStructuresVec = countStructures(currSeq,nStructuresVec,lenSeq)
        nStructures += nStructuresForSeq
        seqInChars = [chars[currSeq[j]] for j in range(lenSeq)]
        #print(seqInChars,nStructuresForSeq) #For testing
        currSeq = makeNextSeq(currSeq)
            
    return(nStructures/(4**lenSeq)) #return average number of structures for a sequence of length lenSeq
        
    
def numPossStructsFaster(lenSeq): #much much faster
    #numPossStructs considers all possible sequences of length lenSeq.
    #But we can do it faster by noting that AAUU and AUAU are not actually different sequences.
    #So we enumerate based on the total number of each ntd in the sequence, and multiply by a factor
    #telling us how many different arrangements of that combination we should expect.
    chars = 'ACGU'
    
    nStructuresVec = np.zeros([lenSeq+1]*4) - 1 #index 0 is number of As, ... 3 is number of Us
    #keep track for each possible sequence the number of possible structures it can form
    
    nStructures = 0
    
    for nAs in range(lenSeq+1): #can have from 0 to lenSeq As
        for nCs in range(max(lenSeq+1 - (nAs),1)): #can have from 0 to (lenSeq - nAs) Cs
            for nGs in range(max(lenSeq+1 - (nAs + nCs),1)): #etc.
                for nUs in [max(lenSeq - (nAs + nCs + nGs),0)]: #once we've decided on nAs, nCs, nGs, we fix nUs 
                    currSeq = [0]*nAs + [1]*nCs + [2]*nGs + [3]*nUs
                    
                    #count number of ways of arranging nAs As, nCs Cs, etc.
                    numWaysOfThisCombination = math.factorial(lenSeq) / (math.factorial(nAs) * math.factorial(nCs) *
                                                                        math.factorial(nGs) * math.factorial(nUs))
                    
                    nStructuresForSeq,nStructuresVec = countStructures(currSeq,nStructuresVec,lenSeq)
                    nStructures += nStructuresForSeq * numWaysOfThisCombination
                    seqInChars = [chars[currSeq[j]] for j in range(lenSeq)]
                    #print(seqInChars,nStructuresForSeq)
            
    return(nStructures/(4**lenSeq)) #return average number of structures for a sequence of length lenSeq


In [15]:
#There are limits to how long you can set the sequences. Each of these take < 1 min but > 1 s
numPossStructs(10)
numPossStructsFaster(20)

15529620.328714823